In [5]:
import tkinter as tk
from threading import Thread
from tkinter import messagebox
import cv2
import os
import time
import numpy as np
import pandas as pd
from datetime import datetime
from ultralytics import YOLO

class ParkingApp:
    def __init__(self, master):
        self.master = master
        master.title("YOLOv8 Parking Space Detector")

        self.label = tk.Label(master, text="Parking Space Detector", font=("Helvetica", 16))
        self.label.pack(pady=10)

        self.start_button = tk.Button(master, text="Start Detection", command=self.start_detection)
        self.start_button.pack(pady=5)

        self.toggle_button = tk.Button(master, text="Toggle Manual Mode", command=self.toggle_manual)
        self.toggle_button.pack(pady=5)

        self.quit_button = tk.Button(master, text="Quit", command=self.quit_program)
        self.quit_button.pack(pady=10)

        self.manual_mode = False
        self.detection_running = False

    def toggle_manual(self):
        self.manual_mode = not self.manual_mode
        print("Manual Mode:", self.manual_mode)

    def start_detection(self):
        if not self.detection_running:
            self.detection_running = True
            self.thread = Thread(target=self.run_yolo_parking)
            self.thread.start()

    def run_yolo_parking(self):
        try:
            model = YOLO('yolov8s.pt')
            with open(r"D:\\CV with AI Project\\yolov8parkingspace-main\\yolov8parkingspace-main\\coco.txt", "r") as my_file:
                class_list = my_file.read().split("\n")

            cap = cv2.VideoCapture(r"D:\\CV with AI Project\\yolov8parkingspace-main\\yolov8parkingspace-main\\parking1.mp4")
            areas = [[(52, 364), (30, 417), (73, 412), (88, 369)],
                     [(105, 353), (86, 428), (137, 427), (146, 358)],
                     [(159, 354), (150, 427), (204, 425), (203, 353)],
                     [(217, 352), (219, 422), (273, 418), (261, 347)],
                     [(274, 345), (286, 417), (338, 415), (321, 345)],
                     [(336, 343), (357, 410), (409, 408), (382, 340)],
                     [(396, 338), (426, 404), (479, 399), (439, 334)],
                     [(458, 333), (494, 397), (543, 390), (495, 330)],
                     [(511, 327), (557, 388), (603, 383), (549, 324)],
                     [(564, 323), (615, 381), (654, 372), (596, 315)],
                     [(616, 316), (666, 369), (703, 363), (642, 312)],
                     [(674, 311), (730, 360), (764, 355), (707, 308)]]

            log_dir = r"D:\\CV with AI Project\\Csv"
            os.makedirs(log_dir, exist_ok=True)
            log_file_path = os.path.join(log_dir, "parking_log_summary.csv")
            detailed_log_path = os.path.join(log_dir, "parking_detailed_log.csv")

            prev_time = time.time()
            frame_counter = 0
            slot_occupancy_counter = [0] * len(areas)
            detailed_log = []

            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                frame_counter += 1
                current_time = time.time()
                fps = 1 / (current_time - prev_time)
                prev_time = current_time

                frame = cv2.resize(frame, (1020, 500))
                results = model.predict(frame)
                detections = results[0].boxes.data
                px = pd.DataFrame(detections).astype("float")

                area_lists = [[] for _ in range(len(areas))]
                total_cars = 0

                for index, row in px.iterrows():
                    x1, y1, x2, y2 = int(row[0]), int(row[1]), int(row[2]), int(row[3])
                    cls_id = int(row[5])
                    cls_name = class_list[cls_id]

                    if 'car' in cls_name:
                        total_cars += 1
                        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

                        for idx, area in enumerate(areas):
                            if cv2.pointPolygonTest(np.array(area, np.int32), (cx, cy), False) >= 0:
                                area_lists[idx].append(cls_name)
                                slot_occupancy_counter[idx] += 1
                                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                                cv2.circle(frame, (cx, cy), 3, (0, 0, 255), -1)
                                cv2.putText(frame, cls_name, (x1, y1 - 5), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)

                for idx, area in enumerate(areas):
                    status = "Free" if len(area_lists[idx]) == 0 else "Occupied"
                    color = (0, 255, 0) if status == "Free" else (0, 0, 255)
                    text_color = (0, 0, 0)
                    bg_color = (144, 238, 144) if status == "Free" else (200, 100, 100)

                    cv2.polylines(frame, [np.array(area, np.int32)], True, color, 2)
                    text_pos = (area[0][0], area[0][1] - 10)
                    label = f"{idx+1}: {status}"

                    (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
                    cv2.rectangle(frame, text_pos, (text_pos[0] + text_width + 4, text_pos[1] + text_height + 4), bg_color, -1)
                    cv2.putText(frame, label, (text_pos[0] + 2, text_pos[1] + text_height + 2),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1)

                free_spaces = sum(1 for l in area_lists if len(l) == 0)
                occupied_spaces = len(areas) - free_spaces

                cv2.putText(frame, f"Free Spaces: {free_spaces}", (20, 30), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 255, 255), 2)
                cv2.putText(frame, f"Occupied Spaces: {occupied_spaces}", (20, 60), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 255, 255), 2)
                cv2.putText(frame, f"FPS: {fps:.2f}", (880, 30), cv2.FONT_HERSHEY_DUPLEX, 0.8, (0, 255, 255), 2)

                log_entry = {
                    "Frame": frame_counter,
                    "Free Spaces": free_spaces,
                    "Occupied Spaces": occupied_spaces,
                    "Total Cars": total_cars,
                }
                for i in range(len(areas)):
                    log_entry[f"Slot {i+1}"] = "Occupied" if len(area_lists[i]) > 0 else "Free"
                detailed_log.append(log_entry)

                cv2.imshow("RGB", frame)
                key = cv2.waitKey(0 if self.manual_mode else 1) & 0xFF
                if key == 27:
                    break

            cap.release()
            cv2.destroyAllWindows()

            df = pd.DataFrame(detailed_log)
            df.to_csv(detailed_log_path, index=False)

            summary = pd.DataFrame({
                "Slot Number": [f"Slot {i+1}" for i in range(len(areas))],
                "Times Occupied": slot_occupancy_counter
            })
            summary.to_csv(log_file_path, index=False)

            print("Logs saved to:", detailed_log_path)
            print("Summary saved to:", log_file_path)

        except Exception as e:
            messagebox.showerror("Error", f"An error occurred:\n{str(e)}")

        self.detection_running = False

    def quit_program(self):
        self.master.quit()

if __name__ == "__main__":
    root = tk.Tk()
    app = ParkingApp(root)
    root.mainloop()


0: 320x640 1 airplane, 4 trucks, 130.3ms
Speed: 3.9ms preprocess, 130.3ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 airplane, 4 trucks, 168.1ms
Speed: 3.2ms preprocess, 168.1ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 airplane, 4 trucks, 145.2ms
Speed: 4.1ms preprocess, 145.2ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 4 trucks, 139.5ms
Speed: 2.5ms preprocess, 139.5ms inference, 1.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 3 trucks, 134.2ms
Speed: 2.1ms preprocess, 134.2ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 1 airplane, 1 bus, 3 trucks, 133.1ms
Speed: 2.5ms preprocess, 133.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 car, 1 airplane, 1 bus, 2 trucks, 140.5ms
Speed: 3.0ms preprocess, 140.5ms inference, 3.0ms postprocess per image at shape (1, 3, 320, 640)

0